# SAM 2024 : TP Requêtes multi-sources pour l'open data

Séances 8 et 9

date du document : 28/03/2024


Objectif : Comprendre et réaliser des jointures entre deux sources réelles distantes lorsque l'accès aux sources est conforme aux standards de l'*open data*.  

Contexte : On considère deux sources réelles ainsi que des exemples de requêtes pouvant être posées sur chacune d'entre elles.

On vous demande de mettre en oeuvre des scénarios de requêtes **MULTI** sources.

Pour 2024 : Faire l'exercice 3 en priorité




- ZHOU Runlin  28717281
- ZHANG Zhile  21201131

## Préparation

**Copier** ce notebook dans votre drive et l'exécuter dans l'environnement colab de google


In [1]:
# !pip install SPARQLWrapper

### Manipuler le résultat d'une requête
Fonctions **tableauResultat** pour extraire  le résultat d'une requête

In [2]:
import pandas as pd

#pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Déterminer la liste des noms d'attributs du résultat, en tenant compte des attributs optionnels.
def schemaResultat(resultats, limit=100):
  d = {}
  for elt in resultats[:limit]:
    for (pos, nom) in enumerate(elt):
      sum_pos, nb = d.get(nom, (0, 0))
      d[nom] = (sum_pos + pos, nb + 1)
  mean_pos_by_name = [(sum_pos / nb, nom) for (nom, (sum_pos, nb)) in d.items()]
  colonnes = [nom for (mean_pos, nom) in sorted(mean_pos_by_name)]
  return colonnes

def tableauResultat(resultats, limit=100, colonnes=None):
  if colonnes is None :
    colonnes = schemaResultat(resultats, limit)
  tableau = []
  for elt in resultats[:limit]:
    row = {name:v.value for name, v in elt.items()}
    tableau.append([row.get(n, "") for n in colonnes])
  return tableau

print ("fonctions définies")

fonctions définies


Afficher le résultat

In [3]:
def afficherResultat(resultats, limit=100):
  print(len(resultats), "éléments")
  colonnes = schemaResultat(resultats, limit)
  print("schéma: ", colonnes)
  result = tableauResultat(resultats, limit, colonnes)
  return pd.DataFrame(result, columns = colonnes)

print ("fonctions définies")

fonctions définies


### Importer le pilote pour accéder aux sources

In [4]:
from SPARQLWrapper import SPARQLWrapper2
print("pilote importé")

pilote importé


### Préparer l'accès à la source BNF

In [5]:
# Connexion à la BNF
BNF = SPARQLWrapper2("http://data.bnf.fr/sparql")

BNF_prefix = """
PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>
"""
print(BNF_prefix)


PREFIX dc: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rda: <http://rdaregistry.info/Elements/w/#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX bnf: <http://data.bnf.fr/ontology/bnf-onto/>

PREFIX bio: <http://vocab.org/bio/0.1/>
PREFIX voc2: <http://rdvocab.info/ElementsGr2/>



### Préparer l'accès à la source DBPEDIA

In [6]:
# Connexion à DBPEDIA France.
DBPEDIA = SPARQLWrapper2("http://fr.dbpedia.org/sparql")

DBPEDIA_prefix =  """
PREFIX dc:   <http://purl.org/dc/terms/>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX dbo:  <http://dbpedia.org/ontology/>

# Dans cette source, les noms de propriétés sont en français
PREFIX dbp:  <http://fr.dbpedia.org/property/>
"""
# print(DBPEDIA_prefix)

## Source 1 : La bibliothèque BNF

### BNF P1: Le nom des personnes
On suppose les relations :
*    **CrééPar**(ouvrage, auteur)
*    **Nom**(auteur, nom)

En SQL, la requête s'exprimerait ainsi:

> select distinct n.nom

> from CrééPar c, Nom n

> where c.auteur = n.auteur

In [7]:
requete = BNF_prefix + """
SELECT distinct ?nom
WHERE {
  ?ouvrage dc:creator ?auteur .       # un ouvrage est créé par au moins un auteur
  ?auteur foaf:name ?nom              # un auteur a un nom
}
LIMIT 5
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
for r in tableauResultat(resultats):
  print(r)

['Claude Baival']
['Musée national du Château de Versailles et de Trianon']
['Henry Bordeaux']
['Paul Jungers']
["École nationale supérieure d'art. Dijon"]


### BNF T1: liste de thèmes
#### Les *themes* des oeuvres dont l'auteur est référencé dans dbpedia france

En SQL, cette requête pourrait s'exprimer sur les tables suivantes

**créé_par** (livre, auteur)

**type_d_expression** (livre, type_expre)  -- exple de tpye d'expression: textuelle

**écrit_en** (livre, langue)

**sujet** (livre, thème)

**auteur_équivalent** (auteur, auteur_source2)

et une condition SQL serait : WHERE auteur_source2 like 'http://fr.dbpedia.org%'

In [8]:
requete = BNF_prefix + """
SELECT distinct ?theme
WHERE {

  # une oeuvre est associée a (au moins) un auteur
  ?livre dc:creator ?auteur .

  # un livre est un ouvrage dont la forme d'expression est textuelle (te)
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # livre écrit en langue française
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a au moins un theme
  ?livre bnf:subject ?theme.

  # l'auteur existe également dans une autre source,
  # on dit qu'il est "aligné" avec un autre instance du même individu.
  ?auteur owl:sameAs ?auteurDbpedia .

  # la référence externe "pointe" vers dbpedia
  FILTER ( regex(?auteurDbpedia, "http://fr.dbpedia.org")  )

}
ORDER BY ?theme
LIMIT 20
"""

BNF.setQuery(requete)
resultats = BNF.query().bindings
afficherResultat(resultats)

# Exemple:
# Informatique
# Langues
# Philosophie

20 éléments
schéma:  ['theme']


,theme
0,Administration publique
1,Anthropologie. Ethnologie
2,Architecture
3,Archéologie. Préhistoire. Histoire ancienne
4,Art
5,Art et science militaires
6,Arts du spectacle
7,Arts graphiques
8,Astronomie
9,Audiovisuel


###  BNF A1 : une liste d'auteurs

A1 : Liste des auteurs de livres écrits en français et nés pendant une certaine période, par exemple au 20eme siècle. Afficher le nom de l'auteur, son année et lieu de naissance.



In [9]:
requete_a1 = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?lieu
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom
?auteur foaf:name ?nom .

# un auteur a une année et un lieu de naissance
?auteur bio:birth ?naissance.
?auteur voc2:placeOfBirth ?lieu

# on sélectionne les années de naissance dans l'intervalle [1900, 2000]
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer
     && xsd:integer (?naissance) < '2000'^^xsd:integer )
}
#ORDER BY ?nom
LIMIT 20
"""

BNF.setQuery(requete_a1)
resultats = BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['nom', 'naissance', 'lieu']


,nom,naissance,lieu
0,Alex Pol Axely,1920,Chaillé-les-Marais (Vendée)
1,Alexandra Pecker,1906,Paris
2,Marie-Françoise Christout,1928,Neuilly-sur-Seine
3,Gabriel Cousin,1918,Droué (Loir et Cher)
4,Éric Muraise,1908,Châteauroux (Indre)
5,Marc Peyre,1904,Marseille
6,Jean Savatier,1922,Poitiers (Vienne
7,Hans Ludwig Cohn Jaffé,1915,"Francfort-sur-le-Main, Allemagne"
8,Jean Canu,1910,"Tourlaville, Manche"
9,Albert Ayme,1920,Montpellier (Hérault)


###  BNF A2 : les auteurs de la BNF référencés dans dbpedia
Les auteurs ayant un identifiant externe pointant vers dbpedia


In [10]:
requete = BNF_prefix + """
SELECT distinct ?nom  ?naissance ?auteurDbpedia
WHERE {

# une oeuvre est associée a (au moins) un auteur
?livre dc:creator ?auteur.

# livre écrit en français
?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

# un auteur a un nom, et une année de naissance
?auteur foaf:name ?nom ; bio:birth ?naissance .

# l'auteur existe également dans une source externe,
# on dit qu'il est "aligné" avec un autre instance du même individu.
?auteur owl:sameAs ?auteurDbpedia .

# naissance dans l'intervalle [1900, 2000] et la référence externe "pointe" vers dbpedia
FILTER (xsd:integer (?naissance) > '1900'^^xsd:integer
     && xsd:integer (?naissance) < '2000'^^xsd:integer
     && regex(?auteurDbpedia, "http://fr.dbpedia.org")
     )
}
ORDER BY desc(?naissance)
LIMIT 10
"""

BNF.setQuery(requete)

resultats = BNF.query().bindings
afficherResultat(resultats)

10 éléments
schéma:  ['nom', 'naissance', 'auteurDbpedia']


,nom,naissance,auteurDbpedia
0,Vickie Gendreau,1989,http://fr.dbpedia.org/resource/Vickie_Gendreau
1,Axl Cendres,1982,http://fr.dbpedia.org/resource/Axl_Cendres
2,Geneviève Castrée,1981,http://fr.dbpedia.org/resource/Genevi%C3%A8ve_Castr%C3%A9e
3,Isabelle Périer,1978,http://fr.dbpedia.org/resource/Isabelle_P%C3%A9rier
4,Nathanaël Dupré La Tour,1977,http://fr.dbpedia.org/resource/Nathana%C3%ABl_Dupr%C3%A9_La_Tour
5,Coralie Delaume,1976,http://fr.dbpedia.org/resource/Coralie_Delaume
6,Claire Guézengar,1972,http://fr.dbpedia.org/resource/Claire_Guezengar
7,Philippe Nassif,1971,http://fr.dbpedia.org/resource/Philippe_Nassif
8,Fatou Cissé,1971,http://fr.dbpedia.org/resource/Fatou_Fanny-Ciss%C3%A9
9,Benoït Violier,1971,http://fr.dbpedia.org/resource/Beno%C3%AEt_Violier


### BNF A3 : Les auteurs d'une oeuvre philosophique

Voir T1 pour la liste des thèmes possibles autres que la philosophie.

On se limite aux oeuvres écrite en français et sont l'auteur est référencé dans dbpedia france.

In [11]:
requete = BNF_prefix + """
SELECT distinct ?nom ?auteurDbpedia ?type_ouvrage
WHERE {

  ?livre dc:creator ?auteur .

  # un livre est un ouvrage textuel
  # (revoir la formulation de ce triplet...)
  # ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le livre a pour theme "Philosophie"
  ?livre bnf:subject "Philosophie" .

  # le nom de l'auteur
  ?auteur foaf:name ?nom .

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER ( regex(?livreWiki, "http://wikidata.org")
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?auteurDbpedia

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats, 5)

20 éléments
schéma:  ['nom', 'auteurDbpedia']


,nom,auteurDbpedia
0,Alexis Carrel,http://fr.dbpedia.org/resource/Alexis_Carrel
1,Antoine de Rivarol,http://fr.dbpedia.org/resource/Antoine_de_Rivarol
2,Bernard-Henri Lévy,http://fr.dbpedia.org/resource/Bernard-Henri_L%C3%A9vy
3,Bernard de Fontenelle,http://fr.dbpedia.org/resource/Bernard_Le_Bouyer_de_Fontenelle
4,Claude Guillon,http://fr.dbpedia.org/resource/Claude_Guillon_%28%C3%A9crivain%29


###  BNF L1 : Les livres avec leurs auteurs

Les livres écrit en français au 21eme siècle,  référencés dans wikidata et dont l'auteur est référencé dans dbpedia

In [12]:
requete = BNF_prefix + """
SELECT distinct ?annee ?nom ?titre ?auteurDbpedia
WHERE {

  ?livre dc:creator ?auteur .

  # l'année de parution du livre
  ?livre dc:date ?annee .

  # le titre du livre
  ?livre dc:title ?titre .

  # un livre est un ouvrage textuel
  ?livre rda:P10004 <http://data.bnf.fr/vocabulary/work-form/te> .

  # le livre existe également dans une autre source
  ?livre owl:sameAs ?livreWiki .

  # livre écrit en français
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .

  # le nom de l'auteur
  ?auteur foaf:name ?nom.

  # l'auteur existe également dans une autre source
  ?auteur owl:sameAs ?auteurDbpedia .

  FILTER (
       xsd:integer (?annee) > '2000'^^xsd:integer
    && xsd:integer (?annee) < '2099'^^xsd:integer
    && regex(?livreWiki, "http://wikidata.org")
    &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
#ORDER BY desc(?annee)

LIMIT 20
"""

BNF.setQuery(requete)
resultats =  BNF.query().bindings
afficherResultat(resultats)

20 éléments
schéma:  ['annee', 'nom', 'titre', 'auteurDbpedia']


,annee,nom,titre,auteurDbpedia
0,2001,Catherine Millet,La vie sexuelle de Catherine M.,http://fr.dbpedia.org/resource/Catherine_Millet
1,2006,Jonathan Littell,Les Bienveillantes,http://fr.dbpedia.org/resource/Jonathan_Littell
2,2006,Pascal Quignard,Villa Amalia,http://fr.dbpedia.org/resource/Pascal_Quignard
3,2001,Claude Simon,Le tramway,http://fr.dbpedia.org/resource/Claude_Simon
4,2001,Michel Houellebecq,Plateforme,http://fr.dbpedia.org/resource/Michel_Houellebecq
5,2015,Michel Houellebecq,Soumission,http://fr.dbpedia.org/resource/Michel_Houellebecq
6,2003,J. M. G. Le Clézio,Révolutions,http://fr.dbpedia.org/resource/J._M._G._Le_Cl%C3%A9zio
7,2001,Yves Bonnefoy,Les planches courbes,http://fr.dbpedia.org/resource/Yves_Bonnefoy
8,2002,André Gide,Le ramier,http://fr.dbpedia.org/resource/Andr%C3%A9_Gide
9,2004,Alain Damasio,La horde du contrevent,http://fr.dbpedia.org/resource/Alain_Damasio


## Source 2 : DBPedia France

### D1 : Informations sur une personne dont le nom contient certains mots.
Afficher le nom, nationalité, profession, date et lieu de naissance

In [13]:
requete_parametree = DBPEDIA_prefix + """
  SELECT ?p ?name ?nationalité ?profession year(xsd:dateTime(?naissance)) as ?naissance  ?lieu
  WHERE {
    ?p a dbo:Person .
    ?p foaf:name ?name .
    ?p dbo:birthDate ?naissance
    optional {?p dbp:nationalité ?nationalité}
    optional {?p dbp:profession ?profession}
    optional {?p dbp:lieuDeNaissance ?lieu}

  FILTER (regex(?p, "{MOTIF}", "i") )
  }
  LIMIT 10
"""

# l'expression régulière du nom à retrouver
motif = "Victor.*Hug"

requete = requete_parametree.replace("{MOTIF}", motif)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'nationalité', 'profession', 'naissance', 'lieu']


,p,name,nationalité,profession,naissance,lieu
0,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
1,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
2,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
3,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
4,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
5,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
6,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
7,http://fr.dbpedia.org/resource/Victor_Hugo_de_Azevedo_Coutinho,Victor Hugo de Azevedo Coutinho,http://fr.dbpedia.org/resource/Portugal,http://fr.dbpedia.org/resource/Officier,1871,http://fr.dbpedia.org/resource/Macao
8,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/New_York
9,http://fr.dbpedia.org/resource/Victor_Hugo_Green,Victor Hugo Green,Américain,,1892,http://fr.dbpedia.org/resource/États-Unis


### D2 : les infos détaillées sur une certaine personne

In [14]:
requete_parametree = DBPEDIA_prefix + """
  SELECT distinct  ?name ?nationalite ?interet ?idee
  WHERE {
    ?pers foaf:name ?name
    optional {?pers dbp:nationalité ?nationalite}
    optional {?pers dbo:mainInterest ?interet}
    optional {?pers dbp:idéesRemarquables ?idee}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
  }
  order by ?interet ?idee
  LIMIT 20
"""
# On fixe une certaine personne
personne = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#personne = 'http://fr.dbpedia.org/resource/Jean-Paul_Sartre'

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", personne)

# print("la requête est :", requete)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

3 éléments
schéma:  ['name', 'interet', 'idee']


,name,interet,idee
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Philosophie,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Politique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Éthique,"Éthiques du féminisme, féminisme existentialiste, éthiques de l'ambigüité"


### D3 : la liste des oeuvres principales d'une personne donnée

In [15]:
requete_parametree = DBPEDIA_prefix +  """
    SELECT distinct ?name ?oeuvre
    WHERE {
      ?pers foaf:name ?name
      optional {?pers  dbp:œuvresPrincipales ?oeuvre}
    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
}
order by ?oeuvre
LIMIT 10
"""
# On fixe une certaine personne
personne = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'

requete = requete_parametree.replace("{PERSONNE}", personne)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

5 éléments
schéma:  ['name', 'oeuvre']


,name,oeuvre
0,Simone de Beauvoir,L'Invitée
1,Simone de Beauvoir,La Force de l'âge
2,Simone de Beauvoir,Le Deuxième Sexe
3,Simone de Beauvoir,Les Mandarins
4,Simone de Beauvoir,Mémoires d'une jeune fille rangée


### D4 : les influenceurs d'une personne donnée

In [16]:
requete_parametree = DBPEDIA_prefix + """
   SELECT distinct ?name ?influenceur
    WHERE {
      ?pers foaf:name ?name
      optional {?pers  dbo:influencedBy ?influenceur}

    FILTER ( ?pers = <{PERSONNE}> && STRLEN(?name) != 0)
}
order by ?influenceur
LIMIT 10
"""

# On fixe une certaine personne
PERSONNE = 'http://fr.dbpedia.org/resource/Simone_de_Beauvoir'
#PERSONNE = 'http://fr.dbpedia.org/resource/Judith_Butler'

# fixer le nom de la personne dans la requête
requete = requete_parametree.replace("{PERSONNE}", PERSONNE)

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
print(len(res), 'resultats')
afficherResultat(res)

10 resultats
10 éléments
schéma:  ['name', 'influenceur']


,name,influenceur
0,Simone de Beauvoir,http://fr.dbpedia.org/resource/Alfred_Adler
1,Simone de Beauvoir,http://fr.dbpedia.org/resource/Edmund_Husserl
2,Simone de Beauvoir,http://fr.dbpedia.org/resource/Emmanuel_Kant
3,Simone de Beauvoir,http://fr.dbpedia.org/resource/François_Poullain_de_La_Barre
4,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Engels
5,Simone de Beauvoir,http://fr.dbpedia.org/resource/Friedrich_Nietzsche
6,Simone de Beauvoir,http://fr.dbpedia.org/resource/Georg_Wilhelm_Friedrich_Hegel
7,Simone de Beauvoir,http://fr.dbpedia.org/resource/Jean-Paul_Sartre
8,Simone de Beauvoir,http://fr.dbpedia.org/resource/Karl_Marx
9,Simone de Beauvoir,http://fr.dbpedia.org/resource/Martin_Heidegger


### D5 : Les auteurs nés en 2001
Utiliser la fonction year pour extraire l'année de naissance à partir d'une date de naissance.

In [17]:
requete_parametree = DBPEDIA_prefix + """
    SELECT distinct ?p ?name year(xsd:dateTime(?naissance)) as ?naissance
    WHERE {
      ?p a dbo:Person .
      ?p foaf:name ?name .
      ?p dbo:birthDate ?naissance
      FILTER( regex(?naissance, "{ANNEE}") && STRLEN(?name) != 0)
    }
    order by ?p
    LIMIT 10
"""

année = 2001
requete = requete_parametree.replace("{ANNEE}", str(année))

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['p', 'name', 'naissance']


,p,name,naissance
0,http://fr.dbpedia.org/resource/Abraham_Attah,Abraham Attah,2001
1,http://fr.dbpedia.org/resource/Adam_Siao_Him_Fa,Adam Siao Him Fa,2001
2,http://fr.dbpedia.org/resource/Adrien_Warion,Adrien Warion,2001
3,http://fr.dbpedia.org/resource/Adèle_Castillon,Adèle Castillon,2001
4,http://fr.dbpedia.org/resource/Ai_Ogura,Ai Ogura,2001
5,http://fr.dbpedia.org/resource/Aiko_de_Toshi,Aiko,2001
6,http://fr.dbpedia.org/resource/Aleksandr_Nikichine,Aleksandr Nikichine,2001
7,http://fr.dbpedia.org/resource/Aleksandr_Smolyar,Aleksandr Smolyar,2001
8,http://fr.dbpedia.org/resource/Aleksej_Pokuševski,Aleksej Pokuševski,2001
9,http://fr.dbpedia.org/resource/Alessandro_Michieletto,Alessandro Michieletto,2001


### D6 : La liste de tous les intérets principaux


In [18]:
requete = DBPEDIA_prefix + """
    SELECT distinct ?interet
    WHERE {
      ?personne dbo:mainInterest ?o .
      ?o rdfs:label ?interet .

      FILTER (langMatches( lang(?interet), "FR" ) )
}
order by ?interet
LIMIT 10
"""

DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
afficherResultat(res)

10 éléments
schéma:  ['interet']


,interet
0,Activisme politique
1,Afrique
2,Agnosticisme
3,Alchimie
4,Amour
5,Amour platonique
6,Angélologie
7,Anthropologie
8,Anthropologie linguistique
9,Anthropologie philosophique


### D7 la liste des philosophes dans Dbpedia

In [19]:
import time

requete_d7 = DBPEDIA_prefix + """
SELECT distinct  ?name ?p
WHERE {
  ?p rdf:type dbo:Philosopher .
  ?p foaf:name ?name .
  FILTER (STRLEN(?name) != 0)
}
order by ?name
limit 10
"""

start_time = time.time()

DBPEDIA.setQuery(requete_d7)
res = DBPEDIA.query().bindings

duree = int ( (time.time() - start_time) * 1000)
print(f"durée {duree} ms")

afficherResultat(res)


durée 86 ms
10 éléments
schéma:  ['name', 'p']


,name,p
0,(),http://fr.dbpedia.org/resource/Alexandre_d'Aphrodise
1,(),http://fr.dbpedia.org/resource/Mélissos
2,(Bienheureux catholique),http://fr.dbpedia.org/resource/Jean_Duns_Scot
3,(Micius),http://fr.dbpedia.org/resource/Mozi
4,(penseur allemand),http://fr.dbpedia.org/resource/Bruno_Bauer
5,(philosophe allemand),http://fr.dbpedia.org/resource/Johann_Gottlieb_Fichte
6,Abd ar-Rahman ibn Nasir as-Sadi au-Tamimi,http://fr.dbpedia.org/resource/Abd_ar-Rahman_ibn_Nasir_as-Sadi
7,Abu Bakr Mohammed ben Abd-el-Malik ben Tufayl el-Qaïci,http://fr.dbpedia.org/resource/Ibn_Tufayl
8,Abu al-Hudhayl,http://fr.dbpedia.org/resource/Abu_al-Hudhayl
9,Abélard,http://fr.dbpedia.org/resource/Pierre_Abélard


# Exercice 1 : Requêtes réparties entre les 2 sources BNF et DBPedia
A compléter

### J1: Jointure A3 avec D2 par boucles imbriquées
Proposer une solution pour calculer la jointure entre A3 sur le site BNF et D2 sur le site DBPedia.
Pour chaque livre de afficher son titre et son année, le nom de l'auteur, sa nationalité et la liste de ses idées remarquables.

In [20]:
# A compléter

# select le titre, annee, et la reference de livre sur BNF
requete_j1 = BNF_prefix + """
SELECT distinct ?auteurDbpedia ?nom ?annee ?titre
WHERE {
  ?livre dc:creator ?auteur .
  ?livre owl:sameAs ?livreWiki .
  ?livre dc:language <http://id.loc.gov/vocabulary/iso639-2/fre> .
  ?livre bnf:subject "Philosophie" .
  ?livre dc:date ?annee .
  ?livre dc:title ?titre .
  ?auteur foaf:name ?nom .
  ?auteur owl:sameAs ?auteurDbpedia .
  FILTER ( regex(?livreWiki, "http://wikidata.org")
       &&  regex(?auteurDbpedia, "http://fr.dbpedia.org")
  )
}
ORDER BY ?nom
"""
BNF.setQuery(requete_j1)
resultats =  BNF.query().bindings

# boucle imbrique, select le nom, la nationalite, l'idee de l'auteur
res_final = []  # pour stocker les lignes de donnee
for res in resultats:
  req = DBPEDIA_prefix + f"""
      SELECT distinct ?name ?nationalite ?idee
      WHERE {{
        ?pers foaf:name ?name
        optional {{?pers dbp:nationalité ?nationalite}}
        optional {{?pers dbp:idéesRemarquables ?idee}}
      FILTER ( ?pers = <{res["auteurDbpedia"].value}> && STRLEN(?name) != 0)
      }}
      GROUP BY ?name """
  DBPEDIA.setQuery(req)
  res_inter = tableauResultat(DBPEDIA.query().bindings)

  # si le resultat est positive
  if len(res_inter) > 0:
    # pour chaque idee différente du même auteur, nous les stockons séquentiellement dans res_final
    for inter in res_inter:
      res_final.append((r, inter))

# affichage du res_final, le premier 20 lignes
for el in res_final[:20]:
  print(el)

(["École nationale supérieure d'art. Dijon"], ['Emmanuel Levinas', "Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité"])
(["École nationale supérieure d'art. Dijon"], ['Gottfried Wilhelm Leibniz', 'Allemand', 'http://fr.dbpedia.org/resource/Calcul_infinitésimal'])
(["École nationale supérieure d'art. Dijon"], ['Gottfried Wilhelm Leibniz', 'Allemand', 'http://fr.dbpedia.org/resource/Compatibilisme'])
(["École nationale supérieure d'art. Dijon"], ['Gottfried Wilhelm Leibniz', 'Allemand', 'http://fr.dbpedia.org/resource/Caractéristique_universelle'])
(["École nationale supérieure d'art. Dijon"], ['Gottfried Wilhelm Leibniz', 'Allemand', "http://fr.dbpedia.org/resource/Principe_d'identité_des_indiscernables"])
(["École nationale supérieure d'art. Dijon"], ['Gottfried Wilhelm Leibniz', 'Allemand', 'http://fr.dbpedia.org/resource/Principe_de_continuité'])
(["École nationale supérieure d'art. Dijon"], ['Gottfried Wilhelm Leibniz', 'Allemand', 'http://fr.dbpedia.org/r

### J2 : Pour une année N afficher la liste des personnes qui sont nées l'année N et qui existent à la fois dans la BNF et dans DBPedia.
Proposer une solution par fusion de deux résultats triés

In [21]:
# A compléter

année = 1894
# les auteurs nes a 2001 dans DBPedia
requete = requete_parametree.replace("{ANNEE}", str(année))
DBPEDIA.setQuery(requete)
res_DBPEDIA = DBPEDIA.query().bindings

# les auteurs nes a 2001 dans BNF
for res in res_DBPEDIA:
    requete_BNF = BNF_prefix + f"""
    SELECT distinct ?nom ?naissance ?auteur
    WHERE {{
        ?auteur owl:sameAs ?auteurDbpedia .
        ?auteur foaf:name ?nom .
        ?auteur bio:birth ?naissance .
    FILTER (xsd:integer (?naissance) = '{année}'^^xsd:integer && ?auteurDbpedia = <{res['p'].value}>)
    }}
    LIMIT 20
    """
    BNF.setQuery(requete_BNF)
    res_BNF = BNF.query().bindings
    if len(res_BNF) > 0:
        print(res_BNF)

### J3 : Quels sont les auteurs d'une oeuvre dont le thème est philosophie (dans la BNF) et qui sont référencés comme étant des philosophes dans DBpedia ?

In [22]:
# A compléter

# on commence par les auteurs d'une oeuvre philosophique
BNF.setQuery(requete_j1)
resultats = BNF.query().bindings

# une jointure avec DBPEDIA
for res in resultats:
  # requete de pour connecter BNF et DBpedia
  requete = DBPEDIA_prefix + f"""
    SELECT distinct ?pers ?name
    WHERE {{
      ?p rdf:type dbo:Philosopher .
      ?pers foaf:name ?name .
    FILTER (?pers = <{res["auteurDbpedia"].value}>)
    }}
    limit 10
  """
  DBPEDIA.setQuery(requete)
  res = tableauResultat(DBPEDIA.query().bindings)
  
  # l'affichage des resultats
  if len(res) > 0:
    print(res)

[['http://fr.dbpedia.org/resource/Emmanuel_Levinas', ''], ['http://fr.dbpedia.org/resource/Emmanuel_Levinas', 'Emmanuel Levinas']]
[['http://fr.dbpedia.org/resource/Gottfried_Wilhelm_Leibniz', 'Gottfried Wilhelm Leibniz'], ['http://fr.dbpedia.org/resource/Gottfried_Wilhelm_Leibniz', '']]
[['http://fr.dbpedia.org/resource/Gottfried_Wilhelm_Leibniz', 'Gottfried Wilhelm Leibniz'], ['http://fr.dbpedia.org/resource/Gottfried_Wilhelm_Leibniz', '']]
[['http://fr.dbpedia.org/resource/Gottfried_Wilhelm_Leibniz', 'Gottfried Wilhelm Leibniz'], ['http://fr.dbpedia.org/resource/Gottfried_Wilhelm_Leibniz', '']]
[['http://fr.dbpedia.org/resource/Jean_Bodin', ''], ['http://fr.dbpedia.org/resource/Jean_Bodin', 'Jean Bodin']]
[['http://fr.dbpedia.org/resource/Karl_Marx', 'Karl Marx'], ['http://fr.dbpedia.org/resource/Karl_Marx', '']]
[['http://fr.dbpedia.org/resource/Nicolas_Malebranche', 'Nicolas Malebranche']]
[['http://fr.dbpedia.org/resource/Nicolas_Malebranche', 'Nicolas Malebranche']]
[['http://fr

# Exercice 2 : Accès aux données par des requêtes de sélections
L'objectif de cet exercice est d'appréhender la situation où l'accès aux données est contraint (voir la notion de [linked data fragment](https://linkeddatafragments.org/concept/)). Le point d'accès n'autorise que des requêtes simples (donc peut coûteuses à traiter) qui ne contiennent que des **sélections**. La clause where d'une requête ne peut contenir qu'**un seul** motif ayant 1 ou 2 variables. Les motifs autorisés sont donc parmi :
*   ?a *propriété* ?b *(cette requête a deux variables)*
*   *A* *propriété* ?b  *(cette requête a une seule variable)*
*   ?a *propriété* *B*  

où ?a et ?b sont des variables et *A*, *B* et *propriété* sont des valeurs fixées avant de poser la requête.

Ainsi, toutes les jointures doivent être exprimées par le client qui accède aux données : vous devez "programmer" toutes les jointures en python.
Pour les requêtes D1 et D6 proposer une solution respectant ce mode d'accès.

In [23]:
# à compléter



# Exercice 3 : Accès par morceau au résultat d'une requête
1) Un utilisateur veut accéder aux données de manière progressive. Proposer une solution pour obtenir le resultat d'une requete par morceau de N tuples, en utilisant les mots-clés OFFSET et LIMIT. Appliquer cela pour lire le résultat de la requête qui affiche le nom des personnes existant dans dbpedia.
Etudier l'influence de l'offset et de la taille du résultat sur le temps de réponse de la requête.

In [24]:
# Exemple d'utilisation de OFFSET et LIMIT

# Compléter cet exemple


import time

requete = DBPEDIA_prefix + """
SELECT distinct ?p ?name
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(STRLEN(?name) != 0)
}
# ORDER BY ?p ?name
offset 10000 limit 10
"""

start_time = time.time()
# exécution de la requête
DBPEDIA.setQuery(requete)
res = DBPEDIA.query().bindings
duree = int ( (time.time() - start_time) * 1000)
print(f"la requête dure {duree} ms")

afficherResultat(res)

la requête dure 92 ms
10 éléments
schéma:  ['p', 'name']


,p,name
0,http://fr.dbpedia.org/resource/Alina_Alexandra_Dumitru,Alina Alexandra Dumitru
1,http://fr.dbpedia.org/resource/Alina_Astafei,Alina Astafei
2,http://fr.dbpedia.org/resource/Alina_Azario,Alina Azario
3,http://fr.dbpedia.org/resource/Alina_Berchakova,Alina Berchakova
4,http://fr.dbpedia.org/resource/Alina_Berezhna,Alina Berezhna
5,http://fr.dbpedia.org/resource/Alina_Dumitrescu,Alina Dumitrescu
6,http://fr.dbpedia.org/resource/Alina_Elizarova,Alina Elizarova
7,http://fr.dbpedia.org/resource/Alina_Freund,Alina Freund
8,http://fr.dbpedia.org/resource/Alina_Fyodorova,Alina Fyodorova
9,http://fr.dbpedia.org/resource/Alina_Goreac,Alina Goreac


2) Définir une fonction traiteRequete(requete) qui retourne un generateur sur une requete de telle sorte que la requete soit lue par morceaux de N tuples et que seuls les morceaux permettant de produire les M lignes demandées par l'utlisateur soit lus. On ne connait pas à l'avance le nombre M de lignes que va lire l'utilisateur. On a M > N.
Indication voir le mot clé python **yield** pour retourner un générateur.

In [46]:
requete = DBPEDIA_prefix + """
SELECT distinct ?name ?p
WHERE {
  ?p rdf:type dbo:Person .
  ?p foaf:name ?name
  FILTER(STRLEN(?name) != 0)
}
"""

def change_offset(requete, offset):
  return requete + f"""offset {offset} limit 10"""
    


# définition d'un générateur
def traiteRequete(requete, offset, max_lire):
  while (max_lire > offset):
    DBPEDIA.setQuery(change_offset(requete, offset))
    res = DBPEDIA.query().bindings
    if len(res) == 0:
      break
    if len(res) > 0:
      print(f"leture d'un morceau de {len(res)} lignes")
      for r in res :
        yield r
    offset += len(res)

# Nombre de tuples que l'utilisateur souhaite lire
a_lire = 20

lu = 0
for a in traiteRequete(requete, 0, a_lire):
  print(a)
  lu += 1
  if(lu >= a_lire):
    break

print()
if(lu < a_lire):
  print(f"Attention !!!! l'utilisateur n'a obtenu que {lu} lignes sur les {a_lire}")
else:
  print(f"OK, l'utilisateur a obtenu {lu} lignes sur les {a_lire}")

leture d'un morceau de 10 lignes
{'name': Value(literal:'Łukasz Bujko'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Bujko')}
{'name': Value(literal:'Łukasz Gibała'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gibała')}
{'name': Value(literal:'Gierak'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Gierak')}
{'name': Value(literal:'Łukasz Grzeszczuk'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Grzeszczuk')}
{'name': Value(literal:'Łukasz Kaczmarek'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Kaczmarek')}
{'name': Value(literal:'Łukasz Klekot'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Klekot')}
{'name': Value(literal:'Łukasz Kohut'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Kohut')}
{'name': Value(literal:'Łukasz Koszarek'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Koszarek')}
{'name': Value(literal:'Łukasz Krupiński'), 'p': Value(uri:'http://fr.dbpedia.org/resource/Łukasz_Krupiński')}
{'name': 

3) Proposer une solution pour accéder de manière progressive au résultat d'une requête de jointure entre 2 sites, par exemple J1.

In [52]:
# à compléter

# definir un nouveau fonction pour genere les requetes rejointe dans la boucle imbrique
def change_pers(param):
  return DBPEDIA_prefix + f"""
    SELECT distinct ?name ?nationalite ?idee
    WHERE {{
      ?pers foaf:name ?name
      optional {{?pers dbp:nationalité ?nationalite}}
      optional {{?pers dbp:idéesRemarquables ?idee}}
    FILTER ( ?pers = <{param}> && STRLEN(?name) != 0)
    }}
    GROUP BY ?name """

# le resultat access a tableau BNF
BNF.setQuery(requete_j1)
res_inter = BNF.query().bindings
# definir un nombre de ligne souhaite a lire
nb_lire = 20
# definir un nombre pour presenter le nombre de ligne lu
nb_lu = 0
# une boucle imbrique pour completer la rejointure
res_final = []  # pour stocker les lignes de donnee
for res in res_inter:
  # affichage des resultat
  for a in traiteRequete(change_pers(res['auteurDbpedia'].value), 0, nb_lire):
    nb_lu += 1
    if nb_lu > nb_lire:
      break
    res_final.append(a)
  else:
    continue
  break

print(f"get {len(res_final)} lignes !")
for el in res_final:
  print(el)

leture d'un morceau de 1 lignes
leture d'un morceau de 10 lignes
leture d'un morceau de 2 lignes
leture d'un morceau de 10 lignes
get 20 lignes !
{'name': Value(literal:'Emmanuel Levinas'), 'idee': Value(literal:"Autre/Autrui, Altérité, Visage, Événement, Éros, Au-delà de l'être, Illéité")}
{'name': Value(literal:'Gottfried Wilhelm Leibniz'), 'nationalite': Value(literal:'Allemand'), 'idee': Value(uri:'http://fr.dbpedia.org/resource/Calcul_infinitésimal')}
{'name': Value(literal:'Gottfried Wilhelm Leibniz'), 'nationalite': Value(literal:'Allemand'), 'idee': Value(uri:'http://fr.dbpedia.org/resource/Compatibilisme')}
{'name': Value(literal:'Gottfried Wilhelm Leibniz'), 'nationalite': Value(literal:'Allemand'), 'idee': Value(uri:'http://fr.dbpedia.org/resource/Caractéristique_universelle')}
{'name': Value(literal:'Gottfried Wilhelm Leibniz'), 'nationalite': Value(literal:'Allemand'), 'idee': Value(uri:"http://fr.dbpedia.org/resource/Principe_d'identité_des_indiscernables")}
{'name': Valu

# Exerice 4 : Cache de requêtes
1) Proposer des améliorations des requêtes précédentes afin d'éviter de poser plusieurs fois une même requête. On suppose que le résultat de la requête ne change pas entre deux invocations de la même requête.

2) Cache avec une capacité limitée : considérer qu'on peut conserver en cache le résultat de C requêtes (C=10) au maximum

In [27]:
# A compléter

# definir une dictionnaire pour simuler la cache, la taille est egale a la capacite de cache
# on stocker les requetes precedent dans la liste, lorsqu'on fait une requete
# --- cas 1: s'il existe une meme requete dans la liste, c-a-d que cette requete est deja stocker dans la cache 
#            donc, on va lire le resultat sur les donnees locales
# --- cas 2: s'il n'existe pas,
#            on a besoin de stocker la requete et le resultat correspondant (comme une paire de cle-valeur)
#            (la stratégie de suppression progressive de la mémoire peut etre defini par FIFO ou LRU ou les autres algorithmes)

# defini une cache
max_taille = 10
cache = {}

def FIFO(ls):
    return ls

def LRU(ls):
    return ls

# la fonction prend la chaine caractere de la requete en parametre
# et retourne une liste de resultat
def try_run_with_cache(req_current):
    # on le trouve dans la cache
    if req_current in cache.keys():
        return cache[req_current]
    # on ne trouve pas
    else:
        if len(cache) == max_taille: # applique l'algo de progressive de la mémoire
            cache = FIFO(cache) 
            # cache = FIFO(cache) 
        else:
            BNF.setQuery(requete_j1)
            res_inter = BNF.query().bindings
            cache.update({req_current:res_inter})


# FAQ: Questions diverses


## Mise au point des requêtes
Pour débugguer et **mettre au point vos requêtes** Sparql, il est utile de les tester directement sur l'interface du point d'accès dédié de la source de données. L'avantage est que cela vous retourne un message indiquant très précisément la nature et l'endroit où se trouve l'erreur dans la requête.
*   [BNF](http://data.bnf.fr/sparql) voir des [exemples](https://api.bnf.fr/sparql-endpoint-de-databnffr)
*   [DBPedia](http://fr.dbpedia.org/sparql) et la liste de tous les [prefixes](http://fr.dbpedia.org/sparql?nsdecl)


## Autres sources de données
Autres sources à votre disposition pour proposer des requêtes multi-sources:

1.   [Mondial](http://www.semwebtech.org/mondial/10/sparql) : données géopolitiques. Attention le résulat des requêtes est en XML pas en JSON.
2.   [Persée](data.persee.fr) : données bibliographiques alignées avec plusieurs autres sources dont dbpedia. Voir des [exemples](http://data.persee.fr/ressources/requetes-exemples/)
3.   [BioPortal](http://sparql.bioontology.org)
4.    [IdRef](https://data.idref.fr/sparql) serveur d’autorité IdRef et les références bibliographiques en provenance du Sudoc.
5.   [Insee](https://rdf.insee.fr/sparql)
6.  [Geonames web services](https://www.geonames.org/export/ws-overview.html)


## SPARKLIS : aide à l'écriture de requêtes Sparql
Pour exprimer vos requêtes en *langage naturel* et les traduire automatiquement en SPARQL, voir l'appli [Sparklis](http://www.irisa.fr/LIS/ferre/sparklis/)


*   onglet **Sparklis** pour exprimer vos requêtes en langage naturel
*   onglet **YASGUI** pour voir la traduction en Sparql



## Nombre d'éléments dans le resultat d'une requête
Pour compter le nombre d'éléments d'une requête, utiliser la notation avec requête imbriquée dans le where :

SELECT (count(1) as ?nb)

WHERE { *sous-requete* }
